In [12]:
import re

with open("data/0A32eTdBKayjCWhZqDOQ.asm", "r", encoding = "ISO-8859-1") as file:
    tokens = re.sub(r'\n|\r|\t',' ',file.read()).split() 

In [14]:
section_prefixes = [
    'HEADER:', '.text:', '.Pav:', '.idata', '.data', '.rdata', '.bss', '.edata:',
    '.rsrc:', '.tls', '.reloc:'
]

In [21]:
op_instr = [
    'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add',
    'imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb'
]

In [16]:
# Platform specific keywords
msft_kw = ['.dll', 'std::', ':dword']

In [17]:
# important keyword to interact with stack and manipulate memories
imp_kw = ['FUNCTION', 'call'] # memcpy_s and memmove_s usually comes after call

In [19]:
filtered = ['HEADER:']
op_list = []

In [23]:
for i in range(1, len(tokens) - 1):
    if any(tokens[i] == op for op in op_instr):
        for op in op_instr:
            filtered.append(op)
            op_list.append(op)
            break
    
    if any(pref in tokens[i] for pref in section_prefixes):
        for pref in section_prefixes:
            if pref in tokens[i]:
                filtered.append(pref)
                break
    
    elif any(kw in tokens[i] for kw in msft_kw):
        filtered.append(tokens[i])
    elif any(tokens[i] == kw for kw in imp_kw):
        two_gram = tokens[i] + ' ' + tokens[i + 1]
        filtered.append(two_gram)
    # subroutine call
    elif tokens[i] == '__stdcall':
        bigram = tokens[i] + ' ' + tokens[i+1].partition("(")[0]
        filtered.append(bigram)
        filtered.append(tokens[i - 1])
    elif tokens[i] == 'db' and tokens[i+1][0] == "'":
        bigram = tokens[i] + ' ' + tokens[i+1]
        filtered.append(bigram)

In [24]:
filtered

['HEADER:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 'call ??0exception@std@@QAE@ABQBD@Z',
 'std::exception::exception(char',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 'call sub_402C51',
 '.text:',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 'call ??3@YAXPAX@Z',
 '.text:',
 'jmp',
 '.text:',
 '.text:',
 '.text:',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:',
 'jmp',
 '.text:

In [ ]:
own_d = {}

def term_freq(r):
    sparse = {}
    df_dic = {}
    
    for w in r[1]:
        if w in own_d:
            if own_d